In [17]:
import pandas as pd

# Cargar el archivo TSV
with open("../data/test.tsv", "r", encoding="utf-8") as file:
    lines = file.readlines()

# Almacenar sentimientos por frase
sentiment_counts = {0: 0, 1: 0, 2: 0}

current_sentiment = None  # Variable para almacenar el sentimiento de la frase

for line in lines:
    line = line.strip()
    
    if line == "":  # Si es una línea vacía, se termina una frase
        if current_sentiment is not None:
            sentiment_counts[current_sentiment] += 1  # Contar solo una vez por frase
        current_sentiment = None  # Reiniciar sentimiento para la nueva frase
    else:
        parts = line.split("\t")
        if len(parts) == 3:
            _, _, sentiment = parts
            current_sentiment = int(sentiment)  # Guardar el sentimiento de la frase actual

# Asegurar que la última frase también se cuenta
if current_sentiment is not None:
    sentiment_counts[current_sentiment] += 1

# Imprimir los resultados
print(f"Cantidad de frases con sentimiento 0 (Negativo): {sentiment_counts[0]}")
print(f"Cantidad de frases con sentimiento 1 (Neutral): {sentiment_counts[1]}")
print(f"Cantidad de frases con sentimiento 2 (Positivo): {sentiment_counts[2]}")

print("Suma de frases: ", sum(sentiment_counts.values()))


Cantidad de frases con sentimiento 0 (Negativo): 5000
Cantidad de frases con sentimiento 1 (Neutral): 5000
Cantidad de frases con sentimiento 2 (Positivo): 0
Suma de frases:  10000


In [28]:
from transformers import pipeline

# Load the pretrained sentiment-analysis pipeline
# By default, the pipeline uses a variant of BERT, called distilbert-base-uncased-finetuned-sst-2-english.
sentiment_analyzer = pipeline("sentiment-analysis")

c:\Users\Stealth\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\Stealth\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Stealth\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded vers

In [14]:
def extraer_balanceado(input_file, output_file, max_por_clase=5000):
    with open(input_file, 'r', encoding='utf-8') as f:
        lineas = f.readlines()

    positivas = []
    negativas = []
    frase_actual = []
    sa_actual = None  # Sentiment Analysis de la frase

    for linea in lineas:
        if linea.strip() == "":
            if frase_actual and sa_actual is not None:
                if sa_actual == '1' and len(positivas) < max_por_clase:
                    positivas.append(frase_actual)
                elif sa_actual == '0' and len(negativas) < max_por_clase:
                    negativas.append(frase_actual)

                # Cortamos si ya tenemos todo
                if len(positivas) == max_por_clase and len(negativas) == max_por_clase:
                    break

            frase_actual = []
            sa_actual = None
        else:
            frase_actual.append(linea)
            partes = linea.strip().split('\t')
            if len(partes) == 3:
                sa = partes[2]
                if sa_actual is None:
                    sa_actual = sa  # Se toma el SA del primer token

    # Guardamos las frases seleccionadas
    with open(output_file, 'w', encoding='utf-8') as f:
        for frase in positivas + negativas:
            f.writelines(frase)
            f.write("\n")  # Línea vacía entre frases

# Ejemplo de uso:
extraer_balanceado("../data/output_test.tsv", "train_balanceado_10000.tsv")


In [18]:
from collections import Counter

# Lista de etiquetas posibles
etiquetas = ['B-ANIM', 'B-BIO', 'B-CEL', 'B-DIS', 'B-EVE', 'B-FOOD', 'B-INST', 'B-LOC', 'B-MEDIA', 'B-MYTH',
             'B-ORG', 'B-PER', 'B-PLANT', 'B-TIME', 'B-VEHI',
             'I-ANIM', 'I-BIO', 'I-CEL', 'I-DIS', 'I-EVE', 'I-FOOD', 'I-INST', 'I-LOC', 'I-MEDIA', 'I-MYTH',
             'I-ORG', 'I-PER', 'I-PLANT', 'I-TIME', 'I-VEHI',
             'O']

# Inicializa el contador
conteo_etiquetas = Counter({etiqueta: 0 for etiqueta in etiquetas})

# Ruta del archivo TSV
ruta_tsv = "../data/train.tsv"  # cambia esto por la ruta de tu archivo

# Leer archivo línea por línea
with open(ruta_tsv, 'r', encoding='utf-8') as f:
    for linea in f:
        partes = linea.strip().split('\t')
        if len(partes) >= 2:
            etiqueta = partes[1]
            if etiqueta in conteo_etiquetas:
                conteo_etiquetas[etiqueta] += 1

# Mostrar el conteo
for etiqueta, cantidad in conteo_etiquetas.items():
    print(f"{etiqueta}: {cantidad}")


B-ANIM: 3914
B-BIO: 45
B-CEL: 672
B-DIS: 3145
B-EVE: 775
B-FOOD: 2672
B-INST: 95
B-LOC: 18008
B-MEDIA: 1821
B-MYTH: 139
B-ORG: 8542
B-PER: 17473
B-PLANT: 2501
B-TIME: 726
B-VEHI: 108
I-ANIM: 1710
I-BIO: 12
I-CEL: 370
I-DIS: 2092
I-EVE: 1288
I-FOOD: 973
I-INST: 96
I-LOC: 8005
I-MEDIA: 2934
I-MYTH: 31
I-ORG: 11205
I-PER: 18473
I-PLANT: 857
I-TIME: 556
I-VEHI: 123
O: 765213
